In [13]:
import itertools
import math
import numpy as np
from bokeh.plotting import figure, show
from bokeh.palettes import d3 as palette
from bokeh.layouts import gridplot
from bokeh.io import output_notebook

import context
from qubitsim.qubit import HybridQubit as hybrid

In [14]:
output_notebook()

Loading BokehJS ...

In [5]:
operating_point, match_freq = 3.0, 10.0
ref_qubit = hybrid.SOSSHybrid(operating_point, match_freq)

In [10]:
def noise_simulation(qubit, ded, time):
    """
    Calculate the process matrix at time with noise ded
    """
    import qubitsim.CJFidelities as CJ
    indices = [0, 1]
    H0 = np.diag(qubit.energies())
    noise = qubit.detuning_noise_qubit(ded)
    process = CJ.CJ(indices, H0, noise, noise_type='quasistatic')
    return process.chi_final_RF(time)


def noise_iteration(qubit, ded_samples, time):
    cj_array = np.zeros((ded_samples.shape[0], 9, 9), dtype=complex)
    for i, ded in enumerate(ded_samples):
        cj_array[i] = noise_simulation(qubit, ded, time)
    return cj_array

In [26]:
ueV_conversion = 0.241799050402417
sigma = 1.0 * ueV_conversion
tfinal = 100.0
max_noise = 1.0
noise_samples, dnoise = np.linspace(-max_noise, max_noise, 8193, retstep=True)
cj_array = noise_iteration(ref_qubit, noise_samples, tfinal)

In [29]:
print(np.max(cj_array.real), np.min(cj_array.real), np.max(cj_array.imag), np.min(cj_array.imag))

0.5000000000000002 0.0 0.0007072714646813028 -0.0007072714646813028


In [34]:
fig_array = []

for i in [0, 4]:
    for j in [0, 4]:
        fig = figure(width=300, height=300, y_range=(np.max(cj_array.real)-1e-6, np.max(cj_array.real)+1e-6))
        fig.line(noise_samples, np.real(cj_array[:, i, j]))
        fig_array.append(fig)
        
grid = gridplot(fig_array, ncols=2)
show(grid)